In [1]:
# display the current IP address
!curl ipecho.net/plain

216.165.95.188

## Information about the storge
1. use google cloud sql
2. 100GB SSD, 3.75G memory, 1 CPU
3. database: PostgreSQL 14.5

## IP Whitelist of the Storage
1. colab (35.0.0.0/8)
2. colab2 (34.0.0.0/8)
3. NYU VPN (216.165.64.0/19), if you want to run at local machine, you can connect to NYU VPN.


If anyone needs to add any other IP to the whitelist, message me directly.

In [2]:
# !pip install psycopg2-binary
# !pip install psycopg2
# !pip install panda
# !pip install sqlalchemy

In [3]:
import psycopg2
import pandas as pd
import sqlalchemy

### connect with psycopg2

In [4]:
print('Connecting to the PostgreSQL database...')
conn = psycopg2.connect(
    host="34.30.45.126",
    port=5432,
    database="finalproject",
    user="postgres",
    password="teamrocket")

# create a cursor
cur = conn.cursor()

Connecting to the PostgreSQL database...


OperationalError: connection to server at "34.30.45.126", port 5432 failed: Connection timed out (0x0000274C/10060)
	Is the server running on that host and accepting TCP/IP connections?


In [ ]:
queryCmd = """SELECT * FROM test_connection"""
cur.execute(queryCmd)
pd.DataFrame(cur.fetchall())

,0
0,success


### connect with sqlalchemy

In [ ]:
# Connect to postgresql by sqlalchemy. The engine is used by dataframe to_sql function
def connect_tcp_socket() -> sqlalchemy.engine.base.Engine:
    """ Initializes a TCP connection pool for a Cloud SQL instance of Postgres. """
    # Note: Saving credentials in environment variables is convenient, but not
    # secure - consider a more secure solution such as
    # Cloud Secret Manager (https://cloud.google.com/secret-manager) to help
    # keep secrets safe.
    db_host = "34.30.45.126"  # e.g. '127.0.0.1' ('172.17.0.1' if deployed to GAE Flex)
    db_user = "postgres"  # e.g. 'my-db-user'
    db_pass = "teamrocket"  # e.g. 'my-db-password'
    db_name = "finalproject"  # e.g. 'my-database'
    db_port = 5432  # e.g. 5432

    pool = sqlalchemy.create_engine(
        # Equivalent URL:
        # postgresql+pg8000://<db_user>:<db_pass>@<db_host>:<db_port>/<db_name>
        sqlalchemy.engine.url.URL.create(
            drivername="postgresql",
            username=db_user,
            password=db_pass,
            host=db_host,
            port=db_port,
            database=db_name,
        ),
        # ...
    )
    return pool

engine = connect_tcp_socket()

In [ ]:
with engine.connect() as conn:
  result = conn.execute(sqlalchemy.text('SELECT * FROM test_connection')).fetchall()
  print(result)

[('success',)]


In [ ]:
# close the communication with the PostgreSQL
cur.close()